# Text Classification (UTS)

## Data Preparation

download libraries & packages yang dibutuhkan

In [ ]:
# Sentence Transformers: Multilingual Sentence, Paragraph, and Image Embeddings using BERT & Co.
!pip install sentence-transformers
!pip install git+https://github.com/MartinoMensio/spacy-sentence-bert.git

# for model paraphrase-xlm-r-multilingual-v1
!pip install https://github.com/MartinoMensio/spacy-sentence-bert/releases/download/v0.1.2/xx_paraphrase_xlm_r_multilingual_v1-0.1.2.tar.gz # xx_paraphrase_xlm_r_multilingual_v1-0.1.2

# for model distiluse-base-multilingual-cased-v2
!pip install https://github.com/MartinoMensio/spacy-sentence-bert/releases/download/v0.1.2/xx_distiluse_base_multilingual_cased_v2-0.1.2.tar.gz #xx_distiluse_base_multilingual_cased_v2-0.1.2

# Stem for Indonesian Language
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/MartinoMensio/spacy-sentence-bert.git to /tmp/pip-req-build-1jptrs2q
  Running command git clone --filter=blob:none --quiet https://github.com/MartinoMensio/spacy-sentence-bert.git /tmp/pip-req-build-1jptrs2q
  Resolved https://github.com/MartinoMensio/spacy-sentence-bert.git to commit 9fb25f92b5be77d42921a66fda2c54caa3199925
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Impor libraries yang dibutuhkan

In [ ]:
import pandas as pd
import numpy as np
from numpy import array


#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# stemming for Indonesian Language
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

nltk.download('punkt')
nltk.download('stopwords')


#for model-building
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler  # to standardize the features
from sklearn.decomposition import PCA  # to apply PCA
import seaborn as sns  # to plot the heat maps


#for word embedding
import gensim
from gensim.models import Word2Vec

# for NLP use BERT
from sentence_transformers import SentenceTransformer
import spacy_sentence_bert

import warnings
warnings.filterwarnings(action = 'ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Persiapan Dataset

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/prosaindata/source/tasks/output/Data_TA.csv")
df = df.dropna()
df = df.reset_index(drop=True)

In [ ]:
df.head(10)

,NPM,Judul,Abstrak,Prodi,Label
0,40411100468,PERANCANGAN DAN IMPLEMENTASI SISTEM DATABASE T...,Sistem informasi akademik (SIAKAD) merupaka...,Teknik Informatika,RPL
1,40411100476,APLIKASI KONTROL DAN MONITORING JARINGAN KOMPU...,Berjalannya koneksi jaringan komputer dengan l...,Teknik Informatika,RPL
2,40411100480,RANCANG BANGUN APLIKASI PROXY SERVER UNTUKENKR...,Web server adalah sebuah perangkat lunak serve...,Teknik Informatika,RPL
3,70411100070,SISTEM PENDUKUNG KEPUTUSAN OPTIMASI PENJADWALA...,Penjadwalan kuliah di Perguruan Tinggi me...,Teknik Informatika,komputasi
4,80411100115,SISTEM AUGMENTED REALITY ANIMASI BENDA BERGERA...,Seiring perkembangan teknologi yang ada diduni...,Teknik Informatika,komputasi
5,70411100007,Gerak Pekerja Pada Game Real Time Strategy Men...,Gerak pekerja ada pada game yang memiliki genr...,Teknik Informatika,komputasi
6,70411100126,RANCANG BANGUN GAME PERAWATAN SAPI KARAPAN MEN...,"Perkembangan game yang semakin pesat, memberik...",Teknik Informatika,komputasi
7,70411100109,EKSTRAKSI FITUR BERBASIS TWO DIMENSIONAL LINEA...,Sistem pengenalan wajah adalah suatu sistem un...,Teknik Informatika,komputasi
8,80411100083,IMPLEMENTASI ALGORITMA PRIM DAN DEPTH FIRST ...,Teknologi mobile game beroperating system open...,Teknik Informatika,komputasi
9,70411100092,Perancangan Sistem Informasi Badan Kepegawaian...,Kantor Badan Kepegawaian kota Bangkalan adalah...,Teknik Informatika,RPL


In [ ]:
label = df["Label"]

In [ ]:
abstract_datas = df["Abstrak"]

In [ ]:
abstract_datas.head(10)

0    Sistem  informasi  akademik  (SIAKAD) merupaka...
1    Berjalannya koneksi jaringan komputer dengan l...
2    Web server adalah sebuah perangkat lunak serve...
3    Penjadwalan  kuliah  di  Perguruan  Tinggi  me...
4    Seiring perkembangan teknologi yang ada diduni...
5    Gerak pekerja ada pada game yang memiliki genr...
6    Perkembangan game yang semakin pesat, memberik...
7    Sistem pengenalan wajah adalah suatu sistem un...
8    Teknologi mobile game beroperating system open...
9    Kantor Badan Kepegawaian kota Bangkalan adalah...
Name: Abstrak, dtype: object

## Preprocessing

In [ ]:
def preprocess(text):
    """convert to lowercase, strip and remove punctuations"""
    text = text.lower() 

    # remove whitespace
    text = text.strip() 
    # compile pattern that have `<.*?>` then replace with ''
    text = re.compile('<.*?>').sub('', text) 

    # compile pattern that have punctuation !"#$%&'()*+, -./:;<=>?@[\]^_`{|}~ then replace with ' '
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)

    # replace all one or more space to one space
    text = re.sub('\s+', ' ', text)

    # replace all number 0-9 to one space
    text = re.sub(r"\d+", "", text)

    # replace all first word or space in the beginning of line to ''
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())

    # replace all digits to one space
    text = re.sub(r'\d',' ',text) 

    # replace all one or more space to one space
    text = re.sub(r'\s+',' ',text) 

    return text

def stopword(string):
    """Stopword removal like I, The, An, be, is, etc"""
    a = [i for i in string.split() if i not in stopwords.words('indonesian')]
    return ' '.join(a)



Bersihkan kata pada abstrak dengan fungsi ```preprocess()``` untuk menghilangkan angka, tanda baca, dan ubah kata ke huruf kecil

In [ ]:
preprocessed_abstract = []
for text in abstract_datas:
    preprocessed_abstract.append(preprocess(str(text)))

In [ ]:
preprocessed_abstract = pd.Series(preprocessed_abstract)

In [ ]:
preprocessed_abstract[90]

'e learning merupakan usaha untuk membuat sebuah transformasi proses belajar mengajar di sekolah dalam bentuk digital yang dijembatani oleh teknologi internet sudah banyak pembelajaran saat ini yang telah didukung oleh e learning namun dalam proses pembelajaran setiap peserta didik mempunyai gaya belajar yang berbeda satu sama lain gaya pembelajaran yang berbeda tersebut dapat dikelompokkan dengan cara menggunakan tool kuisioner dan secara otomatis penelitian ini akan membahas bagaimana menganalisis gaya belajar peserta didik dan mengelompokkannya dengan menggunakan tool kuisioner serta mengklasifikasi gaya belajar peserta didik menggunkan naãve bayes classifier dimensi untuk model gaya belajar yang digunakan adalah felder silvermen learning style model fslsm dengan penelitian ini diharapkan agar dapat mengatasi permasalahan keragaman gaya belajar peserta didik sehingga dapat meningkatkan efektivitas pembelajaran e learning'

Bersihkan kata pada abstrak dengan fungsi ```stopword()``` untuk menghapus kata hubung dan semua kata bantu

In [ ]:
removed_stopword_abstract = []
for text in preprocessed_abstract:
    removed_stopword_abstract.append(stopword(str(text)))

In [ ]:
removed_stopword_abstract = pd.Series(removed_stopword_abstract)

In [ ]:
removed_stopword_abstract[90]

'e learning usaha transformasi proses belajar mengajar sekolah bentuk digital dijembatani teknologi internet pembelajaran didukung e learning proses pembelajaran peserta didik gaya belajar berbeda gaya pembelajaran berbeda dikelompokkan tool kuisioner otomatis penelitian membahas menganalisis gaya belajar peserta didik mengelompokkannya tool kuisioner mengklasifikasi gaya belajar peserta didik menggunkan naãve bayes classifier dimensi model gaya belajar felder silvermen learning style model fslsm penelitian diharapkan mengatasi permasalahan keragaman gaya belajar peserta didik meningkatkan efektivitas pembelajaran e learning'

Bersihkan kata dengan fungsi ```to_base_word()``` untuk mengubah kata menjadi kata dasar.

In [ ]:
# sastrawi
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def to_base_word(string):
    """Change words to base words"""
    string = string.split()
    text = [stemmer.stem(word) for word in string]
    return ' '.join(text)

In [ ]:
removed_stopword_abstract = []
for text in preprocessed_abstract:
    removed_stopword_abstract.append(stopword(str(text)))

cleaned_abstract = []
for text in removed_stopword_abstract:
    cleaned_abstract.append(to_base_word(str(text)))

In [ ]:
cleaned_abstract = pd.Series(cleaned_abstract)

In [ ]:
cleaned_abstract[90]

'e learning usaha transformasi proses ajar ajar sekolah bentuk digital jembatan teknologi internet ajar dukung e learning proses ajar serta didik gaya ajar beda gaya ajar beda kelompok tool kuisioner otomatis teliti bahas analis gaya ajar serta didik kelompok tool kuisioner klasifikasi gaya ajar serta didik gun na ve bayes classifier dimensi model gaya ajar felder silvermen learning style model fslsm teliti harap atas masalah ragam gaya ajar serta didik tingkat efektivitas ajar e learning'

## Word Embedding

```Cosine Simmilarity``` 


Diukur dengan cosinus sudut antara dua vektor dan menentukan apakah dua vektor menunjuk ke arah yang kira-kira sama.

```vector_size (int, optional)``` – Ukuran dimensi vektor kata.


```window (int, optional)``` – Jarak maksimum antara kata yang akan diprediksi dengan kata yang sudah diprediksi.


```min_count (int, optional)``` – Abaikan semua kata yang memiliki total frekuensi sedikit.

### Word embedding use CBOW and Skip Gram model

In [ ]:
# Python program to generate word vectors using Word2Vec
data = []
# iterate through each sentence in the file
for text in cleaned_abstract:
    for i in sent_tokenize(text):
        temp = []
        
        # tokenize the sentence into words
        for j in word_tokenize(i):
            temp.append(j.lower())
    
        data.append(temp)

In [ ]:
data[0]

['sistem',
 'informasi',
 'akademik',
 'siakad',
 'sistem',
 'informasi',
 'fungsi',
 'tangan',
 'kelola',
 'saji',
 'data',
 'data',
 'akademik',
 'fakultas',
 'siakad',
 'anggap',
 'layan',
 'mahasiswa',
 'butuh',
 'informasi',
 'akademik',
 'universitas',
 'trunojoyo',
 'sedia',
 'siakad',
 'database',
 'pusat',
 'sistem',
 'lebih',
 'awat',
 'mudah',
 'butuh',
 'biaya',
 'sistem',
 'potensi',
 'mengahadapi',
 'kendala',
 'kendala',
 'proses',
 'transaksi',
 'data',
 'padat',
 'jaring',
 'database',
 'siakad',
 'lambat',
 'pemrosesan',
 'respon',
 'query',
 'data',
 'simpan',
 'pemrosesan',
 'kompleks',
 'milik',
 'lemah',
 'sedia',
 'data',
 'sistem',
 'kembang',
 'sistem',
 'database',
 'sistem',
 'databases',
 'distribusi',
 'fakultas',
 'jadi',
 'solusi',
 'masalah',
 'basisdata',
 'distribusi',
 'untung',
 'milik',
 'basisdata',
 'pusat',
 'awas',
 'distribusi',
 'reability',
 'availability',
 'cepat',
 'pemrosesan',
 'query',
 'otonomi',
 'local']

Uji coba kata

In [ ]:
first_word1 = "sistem"
first_word2 = "game"
first_word3 = "komputasi"

second_word1 = "perangkat"
second_word2 = "perangkat"
second_word3 = "perangkat"

#### Continous Bag of Word Model

In [ ]:
# Create CBOW model
cbow_model = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100, window = 5)

In [ ]:
result_with_cbow = cbow_model.wv.similarity(first_word1, second_word1)
print(f"Cosine similarity between '{first_word1}' and '{second_word1}' - with CBOW : {result_with_cbow}")

Cosine similarity between 'sistem' and 'perangkat' - with CBOW : 0.967322587966919


In [ ]:
result_with_cbow = cbow_model.wv.similarity(first_word2, second_word2)
print(f"Cosine similarity between '{first_word2}' and '{second_word2}' - with CBOW : {result_with_cbow}")

Cosine similarity between 'game' and 'perangkat' - with CBOW : 0.9849453568458557


In [ ]:
result_with_cbow = cbow_model.wv.similarity(first_word3, second_word3)
print(f"Cosine similarity between '{first_word3}' and '{second_word3}' - with CBOW : {result_with_cbow}")

Cosine similarity between 'komputasi' and 'perangkat' - with CBOW : 0.9893550872802734


#### Skip Gram Model

In [ ]:
# Create Skip Gram model
skip_gram_model = gensim.models.Word2Vec(data, min_count = 1, vector_size = 100,window = 5, sg = 1)

In [ ]:
result_with_cbow = skip_gram_model.wv.similarity(first_word1, second_word1)
print(f"Cosine similarity between '{first_word1}' and '{second_word1}' - with CBOW : {result_with_cbow}")

Cosine similarity between 'sistem' and 'perangkat' - with CBOW : 0.6850606203079224


In [ ]:
result_with_cbow = skip_gram_model.wv.similarity(first_word2, second_word2)
print(f"Cosine similarity between '{first_word2}' and '{second_word2}' - with CBOW : {result_with_cbow}")

Cosine similarity between 'game' and 'perangkat' - with CBOW : 0.7462562322616577


In [ ]:
result_with_cbow = skip_gram_model.wv.similarity(first_word3, second_word3)
print(f"Cosine similarity between '{first_word3}' and '{second_word3}' - with CBOW : {result_with_cbow}")

Cosine similarity between 'komputasi' and 'perangkat' - with CBOW : 0.5140236616134644


## Evaluation

### Evaluation Preparation

Label ```y```

In [ ]:
print(label.unique())

['RPL' 'komputasi' 'PBA']


Data ```X```

In [ ]:
cleaned_abstract

0      sistem informasi akademik siakad sistem inform...
1      jalan koneksi jaring komputer lancar ganggu ha...
2      web server perangkat lunak server berfungsimen...
3      jadwal kuliah guru kompleks masalah variabel t...
4      iring kembang teknologi dunia muncul teknologi...
                             ...                        
805    investasi saham milik resiko rugi dikarenakanp...
806    information retrieval ir ambil informasi simpa...
807    klasifikasi citra proses kelompok piksel citra...
808    identifikasi atribut pejal kaki salah teliti k...
809    topik deteksi objek tarik perhati kembang tekn...
Length: 810, dtype: object

Buat *dataframe* baru dari data yang sudah dibersihkan

In [ ]:
new_data = {"Abstract" : cleaned_abstract, "Label" : label}
new_df = pd.concat(new_data, axis = 1)

In [ ]:
new_df

,Abstract,Label
0,sistem informasi akademik siakad sistem inform...,RPL
1,jalan koneksi jaring komputer lancar ganggu ha...,RPL
2,web server perangkat lunak server berfungsimen...,RPL
3,jadwal kuliah guru kompleks masalah variabel t...,komputasi
4,iring kembang teknologi dunia muncul teknologi...,komputasi
...,...,...
805,investasi saham milik resiko rugi dikarenakanp...,komputasi
806,information retrieval ir ambil informasi simpa...,PBA
807,klasifikasi citra proses kelompok piksel citra...,komputasi
808,identifikasi atribut pejal kaki salah teliti k...,komputasi


Load model dari https://github.com/MartinoMensio/spacy-sentence-bert/ untuk melakukan pelatihan setiap kata dan acuan untuk merubah kata menjadi vektor

In [ ]:
# for model distiluse-base-multilingual-cased-v2
nlp = spacy_sentence_bert.load_model('xx_distiluse_base_multilingual_cased_v2')

Buat kolom ```df['vector']``` dimana setiap abstrak akan direpresentasikan dalam bentuk vektor 

In [ ]:
new_df['vector'] = new_df['Abstract'].apply(lambda x: nlp(x).vector)

In [ ]:
new_df

,Abstract,Label,vector
0,sistem informasi akademik siakad sistem inform...,RPL,"[0.031695776, 0.0023456744, 0.024228357, -0.02..."
1,jalan koneksi jaring komputer lancar ganggu ha...,RPL,"[0.05129607, -0.024710627, 0.007151665, 0.0053..."
2,web server perangkat lunak server berfungsimen...,RPL,"[-0.018094577, 0.008628332, 0.018115591, -0.04..."
3,jadwal kuliah guru kompleks masalah variabel t...,komputasi,"[0.022376861, 0.04639912, 0.023099946, -0.0318..."
4,iring kembang teknologi dunia muncul teknologi...,komputasi,"[0.01736855, 0.012109187, 0.0017237916, -0.039..."
...,...,...,...
805,investasi saham milik resiko rugi dikarenakanp...,komputasi,"[0.069311336, -0.04580292, -0.037823424, -0.02..."
806,information retrieval ir ambil informasi simpa...,PBA,"[0.018961675, 0.058011264, 0.00534609, -0.0410..."
807,klasifikasi citra proses kelompok piksel citra...,komputasi,"[-0.0014633922, 0.0130769415, 0.013453827, -0...."
808,identifikasi atribut pejal kaki salah teliti k...,komputasi,"[0.00845348, 0.016413385, 0.040197726, -0.0150..."


Pisah data X & label y 


*   ```X_train``` train X data 
*   ```X_test``` test X data 


*   ```y_train``` train y data 
*   ```y_test``` test y data 



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(new_df['vector'].tolist(), new_df['Label'].tolist(), test_size=0.33, random_state=42)

### Train with machine learning classifier.

#### Support Vector Classifier(SVM)

#### Gaussian Naive Bayes

In [ ]:
modelGNB = GaussianNB()

modelGNB.fit(X_train, y_train)
y_pred = modelGNB.predict(X_test)
accuracyGNB = accuracy_score(y_pred, y_test)
print("Accuracy Score:", accuracyGNB)

Accuracy Score: 0.7201492537313433


#### K-Nearest Neighbors 


```n_neighbors = 3```

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy Score:", accuracy)

Accuracy Score: 0.8059701492537313


### Classification Test

Uji coba abstrak baru terhadap model yang sudah dilatih


*   ```abstracts``` untuk data 
*   ```labels``` untuk kategori



In [ ]:
abstracts = [
    "Sistem  informasi  akademik  (SIAKAD) merupakan  sistem  informasi  yang  berfungsi  menangani pengelolaan  dan  penyajian  data-data  akademik,  yang  oleh pihak  fakultas  SIAKAD  dianggap  sangat  penting  dalam memberikan  pelayanan  mahasiswa  yang  membutuhkan informasi akademik. Di Universitas Trunojoyo telah tersedia SIAKAD,  namun  masih  menggunakan  database  terpusat. Sistem seperti ini memberikan kelebihan yaitu perawatannya mudah  selain  itu  juga  membutuhkan  sedikit  biaya,  namun sistem  tersebut  juga  berpotensi  mengahadapi  kendala-kendala  yaitu  dalam  proses  transaksi  data  karena  padatnya jaringan yang menuju database SIAKAD, kelambatan dalam pemrosesan  respon  query  dikarenakan  data  yang  tersimpan semakin besar dan pemrosesan semakin kompleks, dan  juga memiliki kelemahan dalam hal ketersediaan data. Untuk  itu sistem  seperti  ini  memerlukan  pengembangan  sistem database  yang  lebih  baik  dengan  menggunakan  sistem databases  terdistribusi  pada  masing-masing  fakultas  yang dapat  dijadikan  solusi  bagi  permasalahan  di  atas.  Karena dalam basisdata terdistribusi terdapat keuntungan yang tidak dimiliki oleh basisdata  terpusat yaitu pengawasan distribusi, reability,  availability,  kecepatan  dalam  pemrosesan  query dan otonomi local.",
    "Berjalannya koneksi jaringan komputer dengan lancar dan tanpa gangguan merupakan harapan seluruh penggunanya,terutama pihak yang bertanggung jawab pada bidang itu, yakni administrator jaringan. Namun tidak selamanya koneksijaringan dapat berjalan dengan lancar tanpa adanya gangguan. Kenyataan di lapangan terutama pada jaringan komputer skalabesar, berpotensi besar terjadi gangguan atau bahkan kerusakan koneksi jaringan komputer.Hal ini rata - rata disebabkan banyaknya penggunaan serta padatnya traffic data dalam jaringan komputer itu sendiri.Oleh karena itu untuk menjaga bahkan meningkatkan kualitas layanan koneksi jaringan maka diperlukan adanya akses kontroldan monitoring pada level administrator jaringan secara real-time berbasis mobile, yang dapat mengatur dan mengetahuiinformasi tentang koneksi jaringan dan dapat diakses setiap saat.Pihak administrator jaringan dituntut untuk terlebih dahulu mengetahui adanya gangguan atau bahkan terputusnyakoneksi jaringan walaupun administrator tidak berada di tempat kontrol dan monitoring, Dengan adanya sistem kontrol danmonitoring berbasis mobile, diharapkan dapat membantu tugas administrator dalam mengontrol dan memantau jaringan sertadalam mengambil kebijakan atas node â node yang terkoneksi pada jaringan yakni memanajemen koneksi, port dan bandwidthsetiap saat dan secara mobile. agar dengan segera dapat melakukan pembenahan ketika terjadi gangguan jaringan sebelumpemakai jaringan (klien) melakukan pengaduan atas adanya gangguan.Kata Kunci : Administrator Jaringan, Koneksi Jaringan, Akses Kontrol dan Monitoring.",
    "Web server adalah sebuah perangkat lunak server yang berfungsimenerima permintaan HTTP atau HTTPS dari web client danmengirimkan jawaban dalam bentuk halaman-halaman web yangumumnya berbentuk dokumen HTML. Dokumen HTML yang dikirimmerupakan dokumen yang dibentuk dari bahasa markup yang plaintext.Artinya, dokumen berisi kode-kode HTML murni tanpa proses enkripsi.Tujuan dari penelitan ini adalah membuat sebuah aplikasi proxyserver yang menerima permintaan HTTP dari web client danmeneruskannya ke web server. Kemudian aplikasi ini akan menerimarespon dari server dan melakukan proses enkripsi pada dokumen HTMLterlebih dahulu. Sehingga dokumen HTML yang dikirim kepada webclient sudah berbentuk dokumen HTML yang terenkripsi. Web clientdisini bisa berupa aplikasi seperti Internet Explorer, Mozilla Firefox,atau Opera. Aplikasi-aplikasi tersebut biasa dikenal dengan sebutan webbrowser.Ketika web browser menerima dokumen HTML yang terenkripsidari aplikasi proxy server, maka JavaScript Engine yang ada di webbrowser akan melakukan proses dekripsi sekaligus rendering sehinggadokumen akan ditampilkan sesuai dengan aslinya. Namun apabila kodedari dokumen HTML tersebut dilihat melalui menu view source yangada di web browser akan didapatkan dokumen HTML yang tetapterenkripsi.",
    "Penjadwalan  kuliah  di  Perguruan  Tinggi  merupakan  masalah yang  kompleks.  Dalam  permasalahan tersebut  terdapat  beberapa variabel  yang  harus  dipertimbangkan  supaya  mendapatkan  jadwal  yang optimal. Di dalam penyusunan jadwal kuliah ini terdapat sangat banyak kemungkinan  yang  selayaknya  dicoba  untuk  menemukan  penjadwalan yang  terbaik.  Karena  itu  dibutuhkan  metode  optimasi  yang  dapat diterapkan  untuk  mengerjakan  penjadwalan  mata  kuliah  ini.  Metode yang  dapat  digunakan  untuk  menyelesaikan  permasalahan  tersebut adalah  dengan  menggunakan  Algoritma  Genetika.  Tugas  Akhir  ini bertujuan  untuk  membuat  suatu  sistem  yang  dapat  membuat  jadwal matakuliah  secara  otomatis  dan  hasilnya  optimal.  Dari  hasil  penelitian yang  dilakukan  algoritma  genetika  mampu  membuat  jadwal  dengan optimal  yang  dibuktikan  dari  nilai  fitness  yang memiliki  nilai  1  secara otomatis  jadwal  yang  dibentuk  akan  optimal.  Semoga  adanya  aplikasi ini  diharapkan  bisa  mempermudah  didalam  pembuatan  penjadwalan matakuliah dengan mudah dan cepat.  Kata Kunci: Algoritma Genetika, Optimasi,  Penjadwalan.",
    "Seiring perkembangan teknologi yang ada didunia, muncul teknologi yang disebut augmented reality. Augmented Reality (AR) merupakan upaya untuk menggabungkan dunia maya dan dunia virtual yang dibuat melalui komputer sehingga batas antara keduanya menjadi sangat tipis. Head-Mounted Display (HMD) merupakan contoh hasil dari penelitian tentang Augmented Reality, ini merupakan satu-satunya peralatan dasar dalam teknologi-teknologi terbaru. Seiring berjalannya waktu, augmented reality berkembang pesat sehingga memungkinkan pengembangan aplikasi ini di berbagai bidang.Untuk mengembangkan AR banyak sekali library pendukung yang dapat digunakan. FlarToolKit adalah salah satunya. FlarToolKit merupakan library pendukung augmented reality pada platform flash.Berdasarkan sistem AR animasi benda bergerak yang dibuat adalah animasi bergerak permainan pingpong dimana telah dikolaborasikan dengan augmented reality untuk menggerakkan setiap langkah dari playernya. Interaksi antar obyek tersebut dapat dilihat dari gerak bola yang dapat dipukul dengan obyek virtual yang ditampilkan melalui augmented reality.",
    "Gerak pekerja ada pada game yang memiliki genre RTS (Real-Time Strategy). Gerak pekerja memiliki berbagai macam gerak. Oleh sebab itu dibutuhkan sebuah pendekatan konsep AI  untuk mendesain perilaku pekerja tersebut. Perilaku karakter tersebut harus ditambahi dengan AI (Artifical intelegent) agar perilakunya menjadi lebih hidup dan realistis.Dalam penelitian ini AI yang digunakan adalah Finite State Machine. Finite State Machine digunakan untuk menentukan gerak pekerja terhadap parameter-parameter yang digunakan sebagai dasar gerak yang akan dilakukan . Selanjutnya akan disimulasikan pada game RTS dengan menggunakan game engine.Hasil yang di peroleh dalam penelitian ini adalah penerapan metode Finite State machine untuk menentukan gerak pekerja berdasarkan parameter jumlah harta, prajurit, kondisi bangunan, dan stockpile (jumlah resources yang di bawa). Kata kunci : Game, Real-Time Strategy, Gerak Pekerja, Finite State Machine.",
    "Perkembangan teknologi komunikasi yang begitu cepat memicu perkembangan teknologi telpon seluler sebagai salah satu alat komunikasi modern. Komunikasi data yang diimplementasikan juga beragam, teks, gambar bahkan suara. Medium yang digunakan-pun kini beragam baik itu internet, local connection hingga bluetooth. Bluetooth dapat dipilih sebagai alternatif medium untuk menghubungkan satu perangkat dengan perangkat lain. Ditambah lagi dengan perkembangan android yang belakangan ini juga semakin pesat semakin memberi kemudahan untuk betukar data melalui bluetooth. Bluetooth mampu menghubungkan perangkat yang satu dengan perangkat lainnya. Ini kemudian memungkinkan antar perangkat dapat saling bertukar atau berkirim data. Namun, dalam proses pengiriman data tersebut tidak ada yang dapat menjamin keamanannya. Hal ini yang kemudian membuat proses enkripsi menjadi hal yang penting. Data yang dikirim terlebih dahulu dikodekan sedemikian rupa hingga hanya dengan kunci tertentu kode tersebut dapat dibaca. RSA kemudian muncul sebagai salah satu alternatif algoritma yang dapat digunakan menangani proses enkripsi tersebut. RSA memiliki teknik pemfaktoran yang cukup rumit, hal ini membuat tingkat keamanannya cukup tinggi. Selain itu, RSA juga cocok diaplikasikan dalam platform mobile karena memiliki struktur yang asimetris sehingga tidak terlalu membebani sistem yang dimiliki oleh perangkat mobile tersebut. Pengujian aplikasi ini sudah teruji melalui perhitungan manual dan dari pengambilan dua device yang berbeda. ",
    "Berita memiliki peranan penting dalam kehidupan manusia. Saat ini tren memperoleh informasi melalui situs berita online semakin populer. Kebanyakan pembaca ingin memperoleh informasi dari sebuah berita dengan cepat, ringkas dan tidak banyak membuang waktu. Tujuan penelitian ini untuk memberikan kemudahan bagi pembaca untuk mendapatkan informasi terpenting dari suatu berita dengan cepat.Pada penelitian ini, dibangun sebuah aplikasi peringkas berita online otomatis yang dikembangkan menggunakan metode perhitungan Ordinary Weighting. Metode Ordinary Weighting merupakan perhitungan bobot term berdasarkan prinsip TF * IDF dengan menggunakan TF tanpa normalisasi. Tujuan aplikasi ini untuk menghasilkan sebuah ringkasan yang terdiri dari tiga kalimat dengan nilai bobot tertinggi sebagai ringkasan dari suatu artikel berita dengan tetap mempertahankan ide utama dari berita tersebut. Proses evaluasi dilakukan dengan cara membandingkan hasil ringkasan yang dihasilkan oleh sistem dengan ringkasan yang dibuat secara manual oleh koresponden melalui metode kuesioner, dengan prinsip Majority Opinion.Dari hasil penelitian diketahui bahwa kesesuaian hasil ringkasan sistem dengan pengguna yang diperoleh dari kuesioner menunjukkan hasil nilai rata-rata recall sebesar 0.47, precision 0.40 dan rata-rata f-measures sebesar 0.53.",
    "Perintah dengan menggunakan suara memungkinkan penggunadapat mengakses isi database dengan mengucapkan perintah melaluiperalatan masukan audio tanpa harus menggunakan cara konvensional.Penggunaan program database dalam pengoperasiannya, diperlukanpemahaman pada struktur SQL (Structured Query Language).Kemudahan bisa dirasakan bagi pengguna awam karena sistempengaksesan database disertai dengan pengembangan NLP (NaturalLanguage Processing).Perintah dengan suara menggunakan piranti masukan audioseperti mikrofon dan mesin pengenal ucapan. Ketika penggunamengucapkan perintah melalui mikrofon, mesin pengenal ucapan akanmenerjemahkan dari ucapan ke teks. Dari teks akan diterjemahkankedalam bentuk perintah SQL untuk pengaksesan database. Penelitianini memanfaatkan Teknologi Pengenalan Suara Automatis padaMicrosoft Speech Aplication Programming Interface. Perintah suarayang diucapkan menggunakan sintaksis bahasa alami, yaitu bahasaInggris. Dengan proses NLP menggunakan metode Top Down Parse.Masalah yang dihadapi dalam menggunakan pengenalanucapan adalah noise juga perlu untuk berbicara sejelas mungkin. Dalampengaplikasiannya dilakukan suatu uji coba pengenalan ucapansebanyak 12 kali pengucapan perintah dan diperoleh hasil keakuratansebesar 33% sedangkan inputan kalimat bahasa alami yang berhasilditerjemahkan dalam bentuk SQL sebanyak 67%.",
]

labels = [
    'RPL',
    'RPL',
    'RPL',
    'komputasi',
    'komputasi',
    'komputasi',
    'PBA',
    'PBA',
    'PBA',
]

Model use KNN, because accuracy score of KNN ($ 0.8 $) is higher than Gaussian Naive Bayes ($ 0.72 $)


In [ ]:
for abstract, label in zip(abstracts, labels):
  print(abstract)
  print(f"True Label: {label}, Predicted Label: {knn.predict(nlp(abstract).vector.reshape(1, -1))[0]} \n")

Sistem  informasi  akademik  (SIAKAD) merupakan  sistem  informasi  yang  berfungsi  menangani pengelolaan  dan  penyajian  data-data  akademik,  yang  oleh pihak  fakultas  SIAKAD  dianggap  sangat  penting  dalam memberikan  pelayanan  mahasiswa  yang  membutuhkan informasi akademik. Di Universitas Trunojoyo telah tersedia SIAKAD,  namun  masih  menggunakan  database  terpusat. Sistem seperti ini memberikan kelebihan yaitu perawatannya mudah  selain  itu  juga  membutuhkan  sedikit  biaya,  namun sistem  tersebut  juga  berpotensi  mengahadapi  kendala-kendala  yaitu  dalam  proses  transaksi  data  karena  padatnya jaringan yang menuju database SIAKAD, kelambatan dalam pemrosesan  respon  query  dikarenakan  data  yang  tersimpan semakin besar dan pemrosesan semakin kompleks, dan  juga memiliki kelemahan dalam hal ketersediaan data. Untuk  itu sistem  seperti  ini  memerlukan  pengembangan  sistem database  yang  lebih  baik  dengan  menggunakan  sistem databases  terdistribusi  pad

***Result***


*   Model KNN, ```n_neigbors = 3```
*   6 Abstrak dapat diprediksi dengan **tepat**
*   4 Abstrak diprediksi dengan **salah**



In [ ]:
for abstract, label in zip(abstracts, labels):
  print(abstract)
  print(f"True Label: {label}, Predicted Label: {modelGNB.predict(nlp(abstract).vector.reshape(1, -1))[0]} \n")

Sistem  informasi  akademik  (SIAKAD) merupakan  sistem  informasi  yang  berfungsi  menangani pengelolaan  dan  penyajian  data-data  akademik,  yang  oleh pihak  fakultas  SIAKAD  dianggap  sangat  penting  dalam memberikan  pelayanan  mahasiswa  yang  membutuhkan informasi akademik. Di Universitas Trunojoyo telah tersedia SIAKAD,  namun  masih  menggunakan  database  terpusat. Sistem seperti ini memberikan kelebihan yaitu perawatannya mudah  selain  itu  juga  membutuhkan  sedikit  biaya,  namun sistem  tersebut  juga  berpotensi  mengahadapi  kendala-kendala  yaitu  dalam  proses  transaksi  data  karena  padatnya jaringan yang menuju database SIAKAD, kelambatan dalam pemrosesan  respon  query  dikarenakan  data  yang  tersimpan semakin besar dan pemrosesan semakin kompleks, dan  juga memiliki kelemahan dalam hal ketersediaan data. Untuk  itu sistem  seperti  ini  memerlukan  pengembangan  sistem database  yang  lebih  baik  dengan  menggunakan  sistem databases  terdistribusi  pad

***Result***


*   Model Gaussian Naive Bayes
*   3 Abstrak dapat diprediksi dengan **tepat**
*   7 Abstrak diprediksi dengan **salah**



## Reduksi Dimensi

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
def split_sequence(sequence, n_steps):
    """
    Split a univariate sequence into samples
        end_ix = i + n_steps # find the end of this pattern
        if end_ix > len(sequence)-1 # check if we are beyond the sequence
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix] # gather input and output parts of the pattern
    """
    X, y = list(), list()
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [ ]:
abstract_vector_column = array(new_df["Abstract"])

In [ ]:
data, data_actual = split_sequence(abstract_vector_column, 3)
matrix_data = pd.DataFrame((data), columns=["Vt-2","Vt-1", "Vt"])

In [ ]:
matrix_data

In [ ]:
#Standardize the features
#Create an object of StandardScaler which is present in sklearn.preprocessing
scalar = StandardScaler()
scaled_data = pd.DataFrame(scalar.fit_transform(matrix_data)) #scaling the data
scaled_data

In [ ]:
#Check the Co-relation between features without PCA
sns.heatmap(scaled_data.corr())